In [1]:
import glob
import pandas as pd
import re
from datetime import datetime
from DSPG_Products import Products

#CSV Dataframes
CSVDataframes = {}

#Saving the files
for file in glob.glob('*.csv'):
    if "master" in file.title().lower(): #Failsafe in case we run it twice and theres already a master CVS
        continue
    df = pd.read_csv(file)
    name = ' '.join(file.title().split()[1:]).split('.')[0]
    CSVDataframes[name] = (df)

#Getting what we need from the Products class
product = Products()
MasterDataFrame = product.ProductDataFrames
for df in MasterDataFrame:
    df.assign(store=None, inplace=True)

# regular expression pattern to match any of the words to remove and capture the word
pattern = re.compile(r'\b(({})\b)'.format('|'.join(map(lambda x: re.escape(x[1]), product.ProductList))), re.IGNORECASE)

#Adding store column to each data frame
for name, data in CSVDataframes.items():
    #Fills in the store
    data['Store'] = pattern.sub("", name)
    #Finds the type of product and adds the dataframe to the corresponding master data frame
    productName = [m[0] for m in pattern.findall(name)]
    index = next((index for index, word in product.ProductList if word in productName[0]), None)
    MasterDataFrame[index] = pd.concat([MasterDataFrame[index], data]).drop_duplicates()

#Saving the master data frames
currentDate = str(datetime(datetime.today().year, datetime.today().month, datetime.today().day))[:-8]
for index, frame in enumerate(MasterDataFrame):
    frame.to_csv("Master " + currentDate + product.ProductList[index][1]+ ".csv", index=False)
